In [20]:
import pandas as pd
import os
import json

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

df = pd.read_csv('input_scenarios/india_base/agent_core_attributes.csv')


In [22]:
df.columns

Index(['state_name', 'sector_abbr', 'geo_sector_n_agents', 'owner_occupancy_status', 'developable_buildings_pct', 'cap_cost_multiplier', 'district_name', 'developable_roof_sqft', 'load_per_customer_in_bin_kwh', 'customers_in_bin', 'state_id', 'district_id', 'tariff_id', 'social_indicator', 'fuzzy_str', 'clean_str', 'centroid', 'load_in_bin_kwh', 'compensation_style', 'tech'], dtype='object')

In [32]:
df.groupby(['sector_abbr'])['customers_in_bin'].sum()

sector_abbr
agg       837679.0
com     70337910.0
ind       798368.0
res    448965369.0
Name: customers_in_bin, dtype: float64

In [34]:
df.groupby(['sector_abbr'])[['developable_roof_sqft', 'load_per_customer_in_bin_kwh']].mean()

,developable_roof_sqft,load_per_customer_in_bin_kwh
sector_abbr,,
agg,21586.625723,2.728279e+06
com,1305.760081,9.405178e+03
ind,14647.537727,4.027070e+06
res,159.334524,7.476737e+02


In [36]:
financing = pd.read_csv('input_scenarios/india_base/financing_rates.csv')
financing.groupby(['sector_abbr'])[['p','q']].describe()

KeyError: "Columns not found: 'q', 'p'"

In [37]:
financing

,state_id,social_indicator,sector_abbr,discount_rate,loan_rate,down_payment
0,35,0.680027,res,0.069253,0.05,0.2
1,28,0.650000,res,0.078128,0.05,0.2
2,12,0.660000,res,0.075172,0.05,0.2
3,18,0.614000,res,0.088768,0.05,0.2
4,10,0.576000,res,0.100000,0.05,0.2
5,4,0.680027,res,0.069253,0.05,0.2
6,22,0.613000,res,0.089064,0.05,0.2
7,26,0.680027,res,0.069253,0.05,0.2
8,25,0.680027,res,0.069253,0.05,0.2
9,30,0.761000,res,0.045320,0.05,0.2


In [43]:
    import numpy as np
    import india.agent_config as config
    import india.helper as helper

    state_id_lookup = pd.read_csv(os.path.join('india','reference_data', 'india_census','state_id_lookup.csv'))
    state_id_lookup = dict(zip(state_id_lookup['state_name'], state_id_lookup['state_id']))

    reeds_load = pd.read_csv(os.path.join('india','reference_data','ReEDS_load.csv'), names=['state','hour','year','value'])

    # --- Group by year ---
    reeds_load = reeds_load.groupby(['state','year'], as_index=False)['value'].mean()

    # --- Pivot Wide ---
    reeds_load = pd.pivot_table(reeds_load, index='state', columns='year', values='value')

    # --- Convert to pct diff ---
    reeds_load = reeds_load.pct_change(axis=1)

    # --- Add missing years ---
    for y in range(2014,2018,1): # not in df
        reeds_load[y] = np.nan

    for y in range(2048,2051,1): # not in df
        reeds_load[y] = np.nan
        
    reeds_load.sort_index(axis=1, inplace=True)

    # --- Add Previous Years --
    reeds_load = reeds_load.fillna(method='bfill', axis=1).fillna(method='ffill', axis=1)

    long_dfs = []
    # --- Calculate cumulative product ---
    for scenario, multiplier in zip(['Low','Planning','High'],[config.LOW_LOAD_GROWTH, config.PLANNING_LOAD_GROWTH, config.HIGH_LOAD_GROWTH]):
        reeds_load += 1
        reeds_load *= multiplier
        reeds_load = reeds_load.cumprod(axis=1)

        # --- Convert back to long_df ---
        load_growth = reeds_load.copy()
        load_growth.reset_index(inplace=True)
        load_growth = load_growth.melt(id_vars=['state'], var_name=['year'], value_name='new_load_growth')

        for c in ['residential', 'commercial', 'industrial', 'agriculture']:
            load_growth[c] = load_growth['new_load_growth'] #TODO: implement sector specific load growth
        load_growth.drop('new_load_growth', axis='columns', inplace=True)

        load_growth = load_growth.melt(id_vars=['state','year'], var_name='sector_abbr', value_name='load_multiplier')
        load_growth['scenario'] = scenario
        long_dfs.append(load_growth)

    load_growth = pd.concat(long_dfs, axis='rows')

    # --- map state_id ---
    load_growth['state'] = load_growth['state'].apply(helper.sanitize_string)
    load_growth['state_id'] = load_growth['state'].map(state_id_lookup)
    load_growth.drop(['state'], axis='columns')
    load_growth['sector_abbr'] = load_growth['sector_abbr'].map({'residential':'res','commercial':'com', 'industrial':'ind', 'agricutlure':'agg'})


FileNotFoundError: [Errno 2] No such file or directory: 'india_base/load_growth_projections.csv'